In [66]:
from bs4 import BeautifulSoup
import csv
import re

In [67]:
html_text ="""<h2>some other text</h2>
<p>text you want to find with keyword</p>
<h1>foo bar foo bar</h1>
hi <h2>text you want to find with keyword</h2>
<a href="someurl">No idea what is going on</a>
hi <div class="gg"> text you want to find with keyword</div>"""

In [ ]:
soup = BeautifulSoup(html_text)


In [ ]:
for elem in soup(text=re.compile(r'\b<h2>\b | \bp\b | \bhhhh\b | \bone_more_maybe\b')):
    print(elem.parent)

In [ ]:
batRegex = re.compile(r'(<[a-z]*>)')
m1=batRegex.search(html_text)

In [ ]:
m1.group(1)

In [ ]:
print (re.findall(r'(<[a-z]*>)',html_text))

In [74]:
batRegex = re.compile(r'([a-zA-Z]{2}\s<[a-zA-Z]*[^>]*>(.+?)</[a-zA-Z]*>)') #\s finds space and newline tooo
m1=batRegex.search(html_text)

In [75]:
m1

<_sre.SRE_Match object; span=(186, 246), match='hi <div class="gg"> text you want to find with ke>

In [76]:
print (re.findall(batRegex,html_text))

[('hi <div class="gg"> text you want to find with keyword</div>', ' text you want to find with keyword')]


In [48]:
l1 = re.findall(batRegex,html_text)
l1[0] = list(l1[0])

In [53]:
l1

[[['hi <div class="gg"> text you want to find with keyword<', 'div>'],
  ' text you want to find with keyword']]

In [52]:
l1[0][0]=l1[0][0].split('/')


In [59]:
l1[0][0][1]=l1[0][0][1].split('>')

In [62]:
keyword = l1[0][0][1][0]

In [63]:
keyword

'div'

In [64]:
l1

[[['hi <div class="gg"> text you want to find with keyword<', ['div', '']],
  ' text you want to find with keyword']]

In [65]:
l1[0][1]

' text you want to find with keyword'

# Naviga


In [ ]:
lists[2][0]
#lists[2][0].split('/')
#lists[2][0][1].split('>')
regex = re.compile(r'\svalue="(.*?)"')
bregex = re.findall(regex,lists[1][0])
bregex
subst = ''
stri =  lists[2][0]
result = stri.replace(bregex[0], subst);
result
    #if(len(res[keyword])==0):
        #res[keyword] = value
    #else:
        #res[keyword].append(value)

# Getting entity types in enriched text

In [ ]:
offset = 0
count = 10
reqparams = {'offset':offset,'count':count}
reqparams['aggr'] = 'term(enriched_text.entities.type,count:30)'
discovery_data = dh_help.getDiscoveryData(discovery_config,reqparams)
print("len of number of docs "+str(len(discovery_data['results'])))
pprint("len of entity types "+str(len(discovery_data['aggregations'][0]['results'])))

ent_len = len(discovery_data['aggregations'][0]['results'])

ent_types = []

for i in range(ent_len):
    pprint(discovery_data['aggregations'][0]['results'][i]['key'])
    ent_types.append(discovery_data['aggregations'][0]['results'][i]['key'])

In [ ]:
ent_types

# Creating soup instance

In [ ]:
source = discovery_data['results'][0]['html']
soup = BeautifulSoup(source, 'lxml')

# Scraping

In [ ]:
res = {}
# dict for 0th doc
res = {}
for tag in soup.findAll('p'):
    #tag.find(x for x in ent_types)
    for ent in ent_types:
        try:
            tag.find(ent)
            res[ent].append(tag.find(ent).text)
        except Exception as e:
            pass

In [ ]:
# for extra tags not in ent_types
for tag in soup.findAll('chron'):
    res['chron'].append(tag.text)

# looping

In [ ]:
result = []
offset = 0
count = 3000
reqparams = {'offset':offset,'count':count}
discovery_data = dh_help.getDiscoveryData(discovery_config,reqparams)

for i in range(3000):
    res={}
    doc = discovery_data["results"][i]
    res["id"]=doc["id"]

    source = discovery_data["results"][i]['html']
    batRegex = re.compile(r'([a-zA-Z0-9]{2}\s<[a-zA-Z]*[^>]*>(.+?)</[a-zA-Z]*>)')

    lists = re.findall(batRegex,source)
    len_ = len(lists)
    for i in range(len_):
        lists[i] = list(lists[i])

    regex = re.compile(r'\svalue="(.*?)"')
    subst = ''
    for i in range(len_):
        bregex = re.findall(regex,lists[i][0])
        if(len(bregex)!=0):
            lists[i][0] = lists[i][0].replace(bregex[0], subst)

    regex = re.compile(r'\sidsrc="(.*?)"')
    for i in range(len_):
        bregex = re.findall(regex,lists[i][0])
        if(len(bregex)!=0):
            lists[i][0] = lists[i][0].replace(bregex[0], subst)

    #res = {}
    for i in range(len_):
        lists[i][0]=lists[i][0].split('/')
        lists[i][0][1]=lists[i][0][1].split('>')
        keyword = lists[i][0][1][0]
        value = lists[i][1]

        if keyword in res:
            if value not in res[keyword]:
                res[keyword].append(value)       
        else:
            res[keyword] = [value]

    result.append(res)